# 错误、调试和测试

## 错误处理

**事先约定返回一个错误代码**

缺点：函数返回结果易于错误码混在一起，要用大量的代码来判断是否为返回结果或出错返回值。

In [14]:
def foo():
    r = some_function()  
    if r==(-1):  #发生错误时，返回(-1)
        return (-1)
    # do something
    return r

def bar():
    r = foo()  
    if r==(-1):  #检测是否有错误
        print('Error')
    else:
        pass

高级语言通常都内置了一套try...except...finally...的错误处理机制。  
优点：  
1. 可以跨越多层调用  


### try

In [15]:
try:
    print('try...')
    r=10/0
    print('result:',r) 
except ZeroDivisionError as e:
    print('except:',e)
finally:
    print('finally...')
print('END')

try...
except: division by zero
finally...
END


当r=10/0发生错误执行时，后续语句不执行，直接跳到except语句。

In [16]:
try:
    print('try...')
    r=10/2
    print('result:',r) #当上式错误执行时，该句不执行，直接跳到except语句
except ZeroDivisionError as e:
    print('except:',e)
finally:
    print('finally...')
print('END')

try...
result: 5.0
finally...
END


当r=10/2正常计算时，except语句不会被执行，finally如果有一定会被执行。

In [17]:
try:
    print('try...')
    r = 10/int('a')
    print('result:',r)
except ValueError as e:
    print('ValueError:',e)
except ZeroDivisionError as e:
    print('ZeroDivisionError:',e)
finally:
    print('finally...')
print('END')

try...
ValueError: invalid literal for int() with base 10: 'a'
finally...
END


ValueError：函数或方法虽然接受了正确的*类型*的参数，但是该参数的*值*不适当。  
比如int('nick')，int函数可以接受字符串类型，但是'nick'字符串不具备表示一个整数的含义。

In [18]:
try:
    print('try...')
    r = 10 / int('2')
    print('result:', r)
except ValueError as e:
    print('ValueError:', e)
except ZeroDivisionError as e:
    print('ZeroDivisionError:', e)
else:
    print('no error!')
finally:
    print('finally...')
print('END')

try...
result: 5.0
no error!
finally...
END


如果没有错误发生，会自动执行else语句。

In [19]:
try:
    r = 10/int('a')
except ValueError as e:
    print('ValueError')
except UnicodeError as e:
    print('UnicodeError')  

ValueError


except永远也捕获不到UnicodeError，因为UnicodeError是ValueError的子类，如果有，也被第一个except给捕获了。  
https://docs.python.org/3/library/exceptions.html#exception-hierarchy  
Exception hierarchy

In [23]:
def foo(s):
    return 10 / int(s)

def bar(s):
    return foo(s) * 2

def main():
    try:
        bar('0')
    except Exception as e:
        print('Error:', e)
    finally:
        print('finally...')
        
main()

Error: division by zero
finally...


main()调用bar(),bar()调用foo(),foo()中出现了错误s=0,此时只要main捕捉到了，就可以处理，即实现了跨越多层捕获错误。

### 调用栈

In [21]:
# err.py:
def foo(s):
    return 10 / int(s)

def bar(s):
    return foo(s) * 2

def main():
    bar('0')

main()

ZeroDivisionError: division by zero

如果错误没有被捕获，它就会一直往上抛直至最后被python解释器捕获，打印一个错误信息，接着退出程序。

### 记录错误

logging模块

In [26]:
import logging 
def foo(s):
    return 10/int(s)
def bar(s):
    return foo(s)*2
def main():
    try:
        bar('0')
    except Exception as e:
        logging.exception(e)
        
main()
print('End')

ERROR:root:division by zero
Traceback (most recent call last):
  File "<ipython-input-26-6b484cf1cede>", line 8, in main
    bar('0')
  File "<ipython-input-26-6b484cf1cede>", line 5, in bar
    return foo(s)*2
  File "<ipython-input-26-6b484cf1cede>", line 3, in foo
    return 10/int(s)
ZeroDivisionError: division by zero


End


### 抛出错误

错误是class，捕获一个错误实际就是捕获到该class的一个实例。

In [28]:
class FooError(ValueError):
    pass

def foo(s):
    n = int(s)
    if n==0:
        raise FooError('invalid value:%s'% s)
    return 10/n

foo('0')

FooError: invalid value:0

提示：只有在必要的时候才定义我们自己的错误类型，尽量使用Python内置的错误类型。

In [29]:
def foo(s):
    n = int(s)
    if n==0:
        raise ValueError('invalid value: %s' % s)
    return 10 / n

def bar():
    try:
        foo('0')
    except ValueError as e:
        print('ValueError!')
        raise

bar()

ValueError!


ValueError: invalid value: 0

为什么打印ValueError后，又把错误raise抛出呢？  
捕获错误相当于记录下来，以便后续追踪。但由于当前函数不知道该如何处理这个错误，所以只能继续往上抛，让顶层调用者去处理。简单解释就是员工解决不了的问题，原样(raise不带任何参数)抛给老板去解决。

In [30]:
try:
    10 / 0
except ZeroDivisionError:
    raise ValueError('input error!')

ValueError: input error!

可改变raise的参数来达到错误转换的目的，但不合逻辑的转换是不行的。

练习：运行下面的代码，根据异常信息进行分析，定位出错误源头，并修复  
<code>
from functools import reduce
def str2num(s):
    return int(s)
def calc(exp):
    ss = exp.split('+')
    ns = map(str2num, ss)
    return reduce(lambda acc, x: acc + x, ns)
def main():
    r = calc('100 + 200 + 345')
    print('100 + 200 + 345 =', r)
    r = calc('99 + 88 + 7.6')
    print('99 + 88 + 7.6 =', r)
main()
</code>

In [52]:
from functools import reduce

def str2num(s):
    try:
        return int(s)
    except:
        return float(s)

def calc(exp):
    ss = exp.split('+') #切分字符串
    ns = map(str2num, ss) #map(function, iterable, ...)
    return reduce(lambda acc, x: acc + x, ns)

def main():
    r = calc('100 + 200 + 345')
    print('100 + 200 + 345 =', r)
    r = calc('99 + 88 + 7.6')
    print('99 + 88 + 7.6 =', r)

main()

100 + 200 + 345 = 645
99 + 88 + 7.6 = 194.6


## 调试

1.print()把变量打印出来看一看

2.断言(assert)

In [56]:
def foo(s):
    n = int(s)
    assert n!=0,'n is zero!'
    return 10/n

def main():
    foo('0')
    
main()

AssertionError: n is zero!

<font color=red>启动Python解释器时可以用-O参数来关闭assert：  
$ python -O err.py  </font>

3.logging

In [66]:
import logging
logging.basicConfig(level=logging.INFO)

s = '0'
n = int(s)
logging.info('n = %d' % n)
print(10 / n)

#需保存为err.py文件
#输入python err.py

ZeroDivisionError: division by zero

<font color=red>应该输出内容：</font>  
$ python err.py  
INFO:root:n = 0  
Traceback (most recent call last):  
  File "err.py", line 8, in <module>  
    print(10 / n)  
ZeroDivisionError: division by zero  

4.pdf

$ python -m pdb err.py  
1 - 查看代码  
n - 单步执行代码  
p 变量名 - 查看变量  
c - 继续运行
q - 结束调试

import pdb  

s = '0'  
n = int(s)  
pdb.set_trace() # 运行到这里会自动暂停  
print(10 / n)  

5.IDE

Visual Studio Code：https://code.visualstudio.com/，需要安装Python插件。

PyCharm：http://www.jetbrains.com/pycharm/

## 单元测试

TDD是测试驱动开发（Test-Driven Development）的英文简称，是敏捷开发中的一项核心实践和技术，也是一种设计方法论。TDD的原理是在开发功能代码之前，先编写单元测试用例代码，测试代码确定需要编写什么产品代码。

## 文档测试